In [22]:
#dataset- Bank support tickets
#importing of all libraries required
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score

In [23]:
#reading the dataset into a pandas dataframe
df=pd.read_csv(r'C:\Users\Swarupa V\Desktop\IT-ticket classification\bank_complaint.csv')
df['ticket'][17]

'I have been reviewing my bank statements and have realized that when I use an ATM card I am being charged as much as {$7.00} - {$10.00} per transaction. I authorize the fee from the ATM so I should see an {$2.00} - {$3.00} fee on top of the money I am withdrawing, but I am also seeing that my bank is charging me a {$2.00} fee for a balance inquiry that I never agreed to and then they charge another {$2.00} for using a non- BOA machine, this is also not disclosed. So depending on the amount taken out at the ATM, these fees could be as much as 20 % of the money withdrawn on a {$40.00} ATM transaction ( {$40.00} ) this is worse than the short term money loans. All of the fees should be disclosed when doing an ATM transaction. My Bank is Bank of America.'

In [24]:
#preprocess text
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def preprocess(s):
    lst=[]
    s=' '.join(s.split()) #removes extra white spaces+ \n,\t
    s=re.sub(r'[^\w]', ' ', s) #removes symbols
    s=s.lower()#to lower case
    s=[ps.stem(x) for x in word_tokenize(s)]#stem words and tokentise it
    s=[x for x in s if not x in stop_words] #remove stop words
    return s

In [25]:
#training the doc2vec model
def doctovec(token_lst):
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(token_lst)]
    doc2vec_model = Doc2Vec(documents, vector_size=300, window=3, min_count=1, workers=4)
    return doc2vec_model
    
    

In [26]:
#write the vectors into a file
def filewrit(token_lst,doc2vec_model):
    doc2vec_filename ='train_review_doc2vec.csv'
    with open(doc2vec_filename, 'w+') as doc2vec_file:
        header = ",".join(str(ele) for ele in range(300))
        doc2vec_file.write(header)
        doc2vec_file.write("\n")
        for row in token_lst:
            model_vector = doc2vec_model.infer_vector(row)
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            doc2vec_file.write(line1)
            doc2vec_file.write('\n')


In [27]:
#calling the functions
complaints=df['ticket']
tokens=[]
for s in complaints:
    tokens.append(preprocess(s))
#we get a list of tokens

    

In [10]:
model=doctovec(tokens)

In [28]:
#writing document vectors into the file
filewrit(tokens,model)

In [29]:
#test
vector = model.infer_vector(tokens[78])
# Printing sample vector
print(len(vector))
print("Top 10 values in Doc2Vec inferred vector:")
print(vector[:10])

300
Top 10 values in Doc2Vec inferred vector:
[ 0.00906968  0.00789744 -0.00528141 -0.00189447 -0.02996049 -0.04532988
  0.00974988  0.0333354  -0.01697093 -0.04704678]


In [52]:
#split into test and train
ddf = pd.read_csv(r'train_review_doc2vec.csv')
x=ddf
y=df['class']
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size = 0.80,random_state=14)
    

In [53]:
from sklearn.tree import DecisionTreeClassifier
clf_decision_doc2vec = DecisionTreeClassifier()
clf_decision_doc2vec.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, clf_decision_doc2vec.predict(X_test)))

Final Accuracy: 0.325


In [54]:
final_model = LogisticRegression(multi_class='multinomial')
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(X_test)))

Final Accuracy: 0.25


In [55]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=4, random_state=14)
clf.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, clf.predict(X_test)))

Final Accuracy: 0.175


In [56]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, knn.predict(X_test)))

Final Accuracy: 0.3125


In [57]:
#Doc2vec creates vectors for large paragraphs or text . Since this model was trained using just 400 samples, the effeciency does not increase much
#however compared to word2vec, this model gives a better accuracy
#KNN stands to be the best classifier with both high and low training data
#Decision tree classifier does equally well  unlike in word2vec